# 01 — Data Cleaning
## Crime Severity Index, Canada (1998–2024)

**Source**: Statistics Canada — Police-reported crime statistics (Crime Severity Index)  
**Raw file**: `../../Raw Data/Severe Data.csv`  
**Output**: `../Clean Data/crime_severity_cleaned.csv`

This notebook loads the raw Statistics Canada export and applies a series of targeted
fixes to produce a clean, analysis-ready dataset with consistent snake_case column names.

In [1]:
# ── Imports + plot style ──────────────────────────────────────────────────────
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# Dark theme matching project style
mpl.rcParams.update({
    'figure.facecolor':  '#0D1117',
    'axes.facecolor':    '#161B22',
    'axes.edgecolor':    '#30363D',
    'axes.labelcolor':   '#E6EDF3',
    'axes.titlecolor':   '#E6EDF3',
    'xtick.color':       '#8B949E',
    'ytick.color':       '#8B949E',
    'text.color':        '#E6EDF3',
    'grid.color':        '#21262D',
    'grid.linestyle':    '--',
    'grid.alpha':        0.5,
    'font.family':       'sans-serif',
    'figure.dpi':        120,
})

RAW_PATH   = '../../Raw Data/Severe Data.csv'
CLEAN_PATH = '../Clean Data/crime_severity_cleaned.csv'

---
## 1 — Load Raw CSV

In [2]:
# ── Load raw CSV ──────────────────────────────────────────────────────────────
df = pd.read_csv(RAW_PATH, dtype=str, encoding='utf-8')

print(f'Shape: {df.shape}')
print()
print('Column dtypes:')
print(df.dtypes)
print()
df.head()

Shape: (22908, 15)

Column dtypes:
REF_DATE         object
GEO              object
DGUID            object
Statistics       object
UOM              object
UOM_ID           object
SCALAR_FACTOR    object
SCALAR_ID        object
VECTOR           object
COORDINATE       object
VALUE            object
STATUS           object
SYMBOL           object
TERMINATED       object
DECIMALS         object
dtype: object



,REF_DATE,GEO,DGUID,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1998,Canada,2021A000011124,Crime severity index,Index,160,units,0,v44312461,1.1,118.84,NaN,NaN,NaN,2
1,1998,Canada,2021A000011124,Violent crime severity index,Index,160,units,0,v44312463,1.3,97.80,NaN,NaN,NaN,2
2,1998,Canada,2021A000011124,Non-violent crime severity index,Index,160,units,0,v44312465,1.5,126.93,NaN,NaN,NaN,2
3,1998,Canada,2021A000011124,Youth crime severity index,Index,160,units,0,v53079455,1.7,110.24,NaN,NaN,NaN,2
4,1998,Canada,2021A000011124,Youth violent crime severity index,Index,160,units,0,v53079457,1.9,86.59,NaN,NaN,NaN,2


---
## 2 — Pre-Clean Quality Report

In [3]:
# ── Pre-clean quality report ──────────────────────────────────────────────────
print('=== Missing / placeholder counts (before cleaning) ===')
print()

# NaN count per column
print('NaN counts:')
print(df.isna().sum())
print()

# Empty-string count per column
empty_counts = (df == '').sum()
print('Empty-string counts:')
print(empty_counts[empty_counts > 0])
print()

# ".." placeholder count per column
dotdot_counts = (df == '..').sum()
print('".." placeholder counts:')
print(dotdot_counts[dotdot_counts > 0])
print()

# Sample rows
print('=== Sample rows ===')
df.sample(5, random_state=42)

=== Missing / placeholder counts (before cleaning) ===

NaN counts:
REF_DATE             0
GEO                  0
DGUID                0
Statistics           0
UOM                  0
UOM_ID               0
SCALAR_FACTOR        0
SCALAR_ID            0
VECTOR               0
COORDINATE           0
VALUE             4620
STATUS           18288
SYMBOL           22908
TERMINATED       22908
DECIMALS             0
dtype: int64

Empty-string counts:
Series([], dtype: int64)

".." placeholder counts:
STATUS    4620
dtype: int64

=== Sample rows ===


,REF_DATE,GEO,DGUID,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
6688,2007,"Victoria, British Columbia [59935]",2021A003559935,Non-violent weighted clearance rate,Number,223,units,0,v53079751,38.17,22.98,NaN,NaN,NaN,2
15629,2017,"Red Deer, Alberta [48830]",2021A003548830,Percent change in youth non-violent crime seve...,Percent,239,units,0,v1593151895,53.12,NaN,..,NaN,NaN,2
16693,2018,"Kelowna, British Columbia [59915]",2021A003559915,Percent change in weighted clearance rate,Percent,239,units,0,v53079859,48.14,0.28,NaN,NaN,NaN,2
3522,2003,Alberta [48],2021A000248,Percent change in violent weighted clearance rate,Percent,239,units,0,v53079701,33.16,-3.69,NaN,NaN,NaN,2
16739,2018,"Abbotsford-Mission, British Columbia [59932]",2021A003559932,Percent change in non-violent crime severity i...,Percent,239,units,0,v44312694,39.6,5.26,NaN,NaN,NaN,2


---
## 3 — Drop Useless Columns

In [4]:
# ── Drop useless columns ──────────────────────────────────────────────────────
# Nearly-empty / encoding-only columns
drop_empty    = ['STATUS', 'SYMBOL', 'TERMINATED']

# Redundant StatsCan metadata
drop_metadata = ['SCALAR_FACTOR', 'SCALAR_ID', 'UOM_ID', 'DGUID', 'COORDINATE']

df = df.drop(columns=drop_empty + drop_metadata)

print(f'Shape after dropping: {df.shape}')
print('Remaining columns:', df.columns.tolist())

Shape after dropping: (22908, 7)
Remaining columns: ['REF_DATE', 'GEO', 'Statistics', 'UOM', 'VECTOR', 'VALUE', 'DECIMALS']


---
## 4 — Fix VALUE: Replace Placeholders → NaN, Cast to float

In [5]:
# ── Replace placeholders in VALUE → NaN, cast to float ───────────────────────
# Raw file uses empty strings (and potentially "..") as missing indicators
df['VALUE'] = df['VALUE'].replace({'..': np.nan, '': np.nan})
df['VALUE'] = df['VALUE'].astype(float)

print(f'VALUE dtype : {df["VALUE"].dtype}')
print(f'VALUE nulls : {df["VALUE"].isna().sum():,}')
print(f'VALUE range : {df["VALUE"].min():.2f} – {df["VALUE"].max():.2f}')

VALUE dtype : float64
VALUE nulls : 4,620
VALUE range : -70.57 – 918.39


---
## 5 — Cast REF_DATE to int

In [6]:
# ── Cast REF_DATE → int (year) ────────────────────────────────────────────────
df['REF_DATE'] = df['REF_DATE'].astype(int)

print(f'REF_DATE dtype : {df["REF_DATE"].dtype}')
print(f'Year range     : {df["REF_DATE"].min()} – {df["REF_DATE"].max()}')

REF_DATE dtype : int32
Year range     : 1998 – 2024


---
## 6 — Strip Province / CMA Codes from GEO

In [7]:
# ── Strip [xx] / [xxxxx] codes from GEO ──────────────────────────────────────
# e.g. "Alberta [48]" → "Alberta"
#      "Calgary, Alberta [48825]" → "Calgary, Alberta"
df['GEO'] = df['GEO'].str.replace(r'\s*\[\d+\]', '', regex=True).str.strip()

print('Unique GEO values (sample):')
print(df['GEO'].unique()[:20])

Unique GEO values (sample):
['Canada' 'Newfoundland and Labrador'
 "St. John's, Newfoundland and Labrador" 'Prince Edward Island'
 'Nova Scotia' 'Halifax, Nova Scotia' 'New Brunswick'
 'Saint John, New Brunswick' 'Quebec' 'Saguenay, Quebec' 'Québec, Quebec'
 'Sherbrooke, Quebec' 'Trois-Rivières, Quebec' 'Montréal, Quebec'
 'Ottawa-Gatineau, Quebec part' 'Ontario'
 'Ottawa-Gatineau, Ontario/Quebec [24505/35505]'
 'Ottawa-Gatineau, Ontario part' 'Toronto, Ontario' 'Hamilton, Ontario']


---
## 7 — Rename Columns to snake_case

In [8]:
# ── Rename columns → snake_case ───────────────────────────────────────────────
rename_map = {
    'REF_DATE':   'year',
    'GEO':        'geography',
    'Statistics': 'metric',
    'UOM':        'unit',
    'VECTOR':     'vector',
    'VALUE':      'value',
    'DECIMALS':   'decimals',
}
df = df.rename(columns=rename_map)

print('Final columns:', df.columns.tolist())

Final columns: ['year', 'geography', 'metric', 'unit', 'vector', 'value', 'decimals']


---
## 8 — Post-Clean Quality Report

In [9]:
# ── Post-clean quality report ─────────────────────────────────────────────────
print('=== Shape ===')
print(df.shape)
print()

print('=== dtypes ===')
print(df.dtypes)
print()

print('=== Missing values ===')
print(df.isna().sum())
print()

# Confirm no ".." remain anywhere
remaining_dotdot = (df.astype(str) == '..').sum().sum()
print(f'Remaining ".." placeholders in any column: {remaining_dotdot}')
print()

print('=== Head ===')
df.head()

=== Shape ===
(22908, 7)

=== dtypes ===
year           int32
geography     object
metric        object
unit          object
vector        object
value        float64
decimals      object
dtype: object

=== Missing values ===
year            0
geography       0
metric          0
unit            0
vector          0
value        4620
decimals        0
dtype: int64

Remaining ".." placeholders in any column: 0

=== Head ===


,year,geography,metric,unit,vector,value,decimals
0,1998,Canada,Crime severity index,Index,v44312461,118.84,2
1,1998,Canada,Violent crime severity index,Index,v44312463,97.80,2
2,1998,Canada,Non-violent crime severity index,Index,v44312465,126.93,2
3,1998,Canada,Youth crime severity index,Index,v53079455,110.24,2
4,1998,Canada,Youth violent crime severity index,Index,v53079457,86.59,2


---
## 9 — Save Cleaned CSV

In [10]:
# ── Save to Clean Data/ ───────────────────────────────────────────────────────
df.to_csv(CLEAN_PATH, index=False, encoding='utf-8')

print(f'Saved: {CLEAN_PATH}')
print(f'Rows : {len(df):,}')
print(f'Cols : {len(df.columns)}')

Saved: ../Clean Data/crime_severity_cleaned.csv


Rows : 22,908
Cols : 7
